In [7]:
import requests
from requests.auth import HTTPBasicAuth
from datetime import datetime

auth_data = HTTPBasicAuth('API-KEY', 'API-SECRET')

date_format = '%Y-%m-%dT%H:%M:%S.%f'

def get_metrics(organization_id):
    api_url = f'https://console.cratedb.cloud/api/v2/organizations/{organization_id}/metrics/prometheus/'
    headers = {'accept': 'plain/text'}
    response = requests.get(api_url, auth=auth_data)
    return response.text

def num_shards(metrics):
    for row in metrics.split('\n'):
        line = row.split(' ')
        if line[0].startswith('crate_node{name="shard_stats"') and 'property="total"' in line[0] and 'crate-data-hot-97ebb337-b44e-446f-adca-a092fd1df0aa' in line[0]:
            return float(line[1])

def get_cluster_status(cluster_id):
    api_url = f'https://console.cratedb.cloud/api/v2/clusters/{cluster_id}/'
    return requests.get(api_url, auth=auth_data).json()

def get_cluster_running_op(cluster_status, cluster_id):
    if (datetime.now() - datetime.strptime(cluster_status['health']['last_seen'], date_format)).seconds > 30:
        cluster_status = get_cluster_status(cluster_id)
    return cluster_status['health']['running_operation']

def get_cluster_num_nodes(cluster_status, cluster_id):
    if (datetime.now() - datetime.strptime(cluster_status['health']['last_seen'], date_format)).seconds > 30:
        cluster_status = get_cluster_status(cluster_id)
    return cluster_status['num_nodes'] - 1


In [9]:
import time

delay_seconds = 5
max_num_shard = 30

organization_id = 'Use your organisation ID here'
cluster_id = 'Use your Cluster ID here'

api_url = f'https://console.cratedb.cloud/api/v2/clusters/{cluster_id}/scale/'

while True:
    metrics = get_metrics(organization_id)  # Pass organization_id to get_metrics function
    cluster_status = get_cluster_status(cluster_id)
    num = num_shards(metrics)
    if num is not None:  # Check if num is not None
        print(f'Current number of shards: {num}')
        if num > (0.8 * max_num_shard):
            num_nodes = get_cluster_num_nodes(cluster_status, cluster_id)  # Pass cluster_id to get_cluster_num_nodes function
            print(f'Start scaling out from {num_nodes + 1} to {num_nodes + 2}')
            requests.put(api_url, json={'product_unit':num_nodes + 1}, auth=auth_data)
            time.sleep(delay_seconds)
            while get_cluster_running_op(cluster_status, cluster_id) != '':
                cluster_status = get_cluster_status(cluster_id)
                continue
            print('Scaled up successfully!')
                
        elif num < (0.5 * max_num_shard):
            num_nodes = get_cluster_num_nodes(cluster_status, cluster_id)  # Pass cluster_id to get_cluster_num_nodes function
            print(f'Start scaling down from {num_nodes + 1} to {num_nodes}')
            requests.put(api_url, json={'product_unit':num_nodes - 1}, auth=auth_data)
            time.sleep(delay_seconds)
            while get_cluster_running_op(cluster_status, cluster_id) != '':
                cluster_status = get_cluster_status(cluster_id)
                continue
            print('Scaled down successfully!')
        else:
            print('Nothing to do!')
    else:
        print('Failed to retrieve shard metrics.')
    time.sleep(delay_seconds)



Current number of shards: 25.0
Start scaling out from 3 to 4
